In [24]:
from datetime import datetime
from pathlib import Path

from pydantic_ai import Agent
from pydantic_ai import messages as _messages
from pydantic_graph import End

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
agent = Agent(model="google-gla:gemini-2.0-flash")


@agent.tool_plain
def get_current_date() -> str:
    "Get today's date"

    return datetime.now().strftime("%d-%m-%Y")


date_formatter = Agent(
    model="google-gla:gemini-2.0-flash",
    system_prompt=(
        "Given a date in 'dd-mm-yyyy' format, reformat it to plain english.\n"
        "For example: 01-01-2025 -> January first, two thousand and twenty five"
    ),
)

In [28]:
async with agent.iter(user_prompt="is tomorrow valentine's day?") as agent_run:
    node = agent_run.next_node
    all_nodes = [node]
    node_index = 1
    while not isinstance(node, End):
        node = await agent_run.next(node)
        if agent.is_model_request_node(node) and agent.is_call_tools_node(all_nodes[node_index - 1]):
            for part in node.request.parts:
                if isinstance(part, _messages.ToolReturnPart) and part.tool_name == "get_current_date":
                    part.content = (await date_formatter.run(user_prompt=f"Date: {part.content}")).data
        all_nodes.append(node)
        node_index += 1

In [29]:
print(agent_run.result.data)

No, tomorrow is March 13, 2025, so it is not Valentine's Day. Valentine's Day is on February 14th.



In [30]:
agent_run.ctx.state.message_history

[ModelRequest(parts=[UserPromptPart(content="is tomorrow valentine's day?", timestamp=datetime.datetime(2025, 3, 11, 19, 20, 23, 776116, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='get_current_date', args={}, tool_call_id=None, part_kind='tool-call')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 3, 11, 19, 20, 26, 103994, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='get_current_date', content='March twelfth, two thousand and twenty five\n', tool_call_id=None, timestamp=datetime.datetime(2025, 3, 11, 19, 20, 26, 105605, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request'),
 ModelResponse(parts=[TextPart(content="No, tomorrow is March 13, 2025, so it is not Valentine's Day. Valentine's Day is on February 14th.\n", part_kind='text')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 3, 11, 19, 20, 27, 7857

In [27]:
Path("val.json").write_bytes(
    _messages.ModelMessagesTypeAdapter.dump_json(agent_run.ctx.state.message_history, indent=2)
)

1091

In [31]:
all_nodes

[UserPromptNode(user_prompt="is tomorrow valentine's day?", system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={}),
 ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content="is tomorrow valentine's day?", timestamp=datetime.datetime(2025, 3, 11, 19, 20, 23, 776116, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request')),
 CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='get_current_date', args={}, tool_call_id=None, part_kind='tool-call')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 3, 11, 19, 20, 26, 103994, tzinfo=datetime.timezone.utc), kind='response')),
 ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='get_current_date', content='March twelfth, two thousand and twenty five\n', tool_call_id=None, timestamp=datetime.datetime(2025, 3, 11, 19, 20, 26, 105605, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request')),
 CallToolsNode(model_resp